In [ ]:
###### Inferência Regressão Logística ######

import requests
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, log_loss

df = catalog.load('download_data_prod')

In [46]:
df

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
10,Jump Shot,Jump Shot,309,20000012,33.8063,-94,238,-118.3638,1,3,...,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,11
17,Jump Shot,Jump Shot,138,20000019,33.8183,-117,226,-118.3868,8,2,...,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-11-01,LAL vs. UTA,UTA,18
27,Jump Shot,Jump Shot,369,20000019,33.8583,-183,186,-118.4528,0,3,...,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-11-01,LAL vs. UTA,UTA,28
39,Jump Shot,Jump Shot,202,20000047,33.7723,-27,272,-118.2968,0,2,...,3PT Field Goal,Center(C),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-11-04,LAL @ VAN,VAN,40
54,Jump Shot,Jump Shot,60,20000049,33.7973,-141,247,-118.4108,4,1,...,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-11-05,LAL vs. LAC,LAC,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30675,Jump Shot,Jump Shot,84,49900088,33.8283,-134,216,-118.4038,2,1,...,3PT Field Goal,Left Side Center(LC),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30676
30681,Jump Shot,Jump Shot,218,49900088,33.7833,-18,261,-118.2878,0,2,...,3PT Field Goal,Center(C),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30682
30685,Jump Shot,Jump Shot,249,49900088,33.7943,81,250,-118.1888,7,3,...,3PT Field Goal,Center(C),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30686
30689,Jump Shot,Jump Shot,326,49900088,33.3653,-12,679,-118.2818,0,3,...,3PT Field Goal,Back Court(BC),Backcourt,Back Court Shot,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30690


In [47]:
features = df[['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance', 'shot_made_flag']]
features_clean = features.dropna(subset=['shot_made_flag'])
features_clean

,lat,lon,minutes_remaining,period,playoffs,shot_distance,shot_made_flag
10,33.8063,-118.3638,1,3,0,25,0.0
17,33.8183,-118.3868,8,2,0,25,1.0
27,33.8583,-118.4528,0,3,0,26,0.0
39,33.7723,-118.2968,0,2,0,27,1.0
55,33.8503,-118.4248,2,1,0,24,0.0
...,...,...,...,...,...,...,...
30675,33.8283,-118.4038,2,1,1,25,0.0
30681,33.7833,-118.2878,0,2,1,26,1.0
30685,33.7943,-118.1888,7,3,1,26,0.0
30689,33.3653,-118.2818,0,3,1,67,0.0


In [52]:
rows = features_clean.drop(columns=['shot_made_flag']).values.tolist()
column_names = ['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance']

# Requisição com dataframe_split
resp = requests.post(
    'http://localhost:5600/invocations',
    json={
        'dataframe_split': {
            'columns': column_names,
            'data': rows
        }
    }
)

# print(resp.json())

# Extrair previsões
predictions = resp.json()['predictions']
real_values = features_clean['shot_made_flag'].tolist()

# Verificar se as previsões são probabilidades ou apenas classes (0 ou 1)
if isinstance(predictions[0], list):  # Exemplo: [[0.1, 0.9], [0.8, 0.2], ...]
    y_pred_proba = np.array(predictions)[:, 1]  # Pegar a probabilidade da classe 1
else:
    y_pred_proba = np.array(predictions)  # Se for apenas um array, assume que é a previsão direta

# Comparar
hits = sum(1 for real, pred in zip(real_values, predictions) if real == pred)
accuracy = accuracy_score(real_values, predictions)
f1 = f1_score(real_values, predictions)
logloss = log_loss(real_values, y_pred_proba)

print(f"Número de acertos: {hits} de {len(real_values)}")
print(f"Acurácia: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Log loss: {logloss / 100:.2f}")

Número de acertos: 3630 de 5412
Acurácia: 0.67
F1 Score: 0.00
Log loss: 0.12


In [55]:
# Teste utilizando a normalização dos dados com RobustScaler

from sklearn.preprocessing import RobustScaler

req_rows = features_clean.drop(columns=['shot_made_flag'])

# Aplicar RobustScaler
scaler = RobustScaler()
scaled = scaler.fit_transform(req_rows)
scaled_list = scaled.tolist()


column_names = ['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance']

# Requisição com dataframe_split
resp = requests.post(
    'http://localhost:5600/invocations',
    json={
        'dataframe_split': {
            'columns': column_names,
            'data': scaled_list
        }
    }
)

# print(resp.json())

# Extrair previsões
predictions = resp.json()['predictions']
real_values = features_clean['shot_made_flag'].tolist()

# Verificar se as previsões são probabilidades ou apenas classes (0 ou 1)
if isinstance(predictions[0], list):  # Exemplo: [[0.1, 0.9], [0.8, 0.2], ...]
    y_pred_proba = np.array(predictions)[:, 1]  # Pegar a probabilidade da classe 1
else:
    y_pred_proba = np.array(predictions)  # Se for apenas um array, assume que é a previsão direta

# Comparar
hits = sum(1 for real, pred in zip(real_values, predictions) if real == pred)
accuracy = accuracy_score(real_values, predictions)
f1 = f1_score(real_values, predictions)
logloss = log_loss(real_values, y_pred_proba)

print(f"Número de acertos: {hits} de {len(real_values)}")
print(f"Acurácia: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Log loss: {logloss / 100:.2f}")

Número de acertos: 3425 de 5412
Acurácia: 0.63
F1 Score: 0.20
Log loss: 0.13
